# Analyzing Trajectories

## Overview

### Questions

* How can I analyze trajectories?

### Objectives

* Describe how to access trajectory frames in **GSD**.
* Examine the trajectory with **freud** and **fresnel**.

## Boilerplate code

In [ ]:
import gsd.hoomd
import fresnel
import math
import freud
import matplotlib
%matplotlib inline
matplotlib.style.use('ggplot')

The `render` function in the next (hidden) cell will render a snapshot using **fresnel**.
`render_movie` will render a sequence of frames as an animated GIF.
These methods accept a *particles* argument that filters out which particles to display.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel, PIL, IPython, warnings, io, numpy

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=250, h=250)

def render(snapshot, particles=None, is_solid=None):
    N = snapshot.particles.N
    L = snapshot.configuration.box[0]
    if particles is not None:
        N = len(particles)
    if is_solid is not None:
        N = int(numpy.sum(is_solid))
    
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=N,
                                       radius=0.5)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    if particles is None and is_solid is None:
        geometry.position[:] = snapshot.particles.position[:]
    elif particles is not None:
        geometry.position[:] = snapshot.particles.position[particles, :]
    elif is_solid is not None:
        geometry.position[:] = snapshot.particles.position[numpy.ix_(is_solid, [0,1,2])]
        
        
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.Orthographic(position=(0, 0, L+1),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L+1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=100)

def render_movie(frames, particles=None, is_solid=None):
    if is_solid is None:
        is_solid = [None]*len(frames)
    a = render(frames[0], particles, is_solid[0]);

    im0 = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB').convert("P", palette=PIL.Image.ADAPTIVE);
    ims = [];
    for i,f in enumerate(frames[1:]):
        a = render(f, particles, is_solid[i]);
        im = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB')
        im_p = im.quantize(palette=im0);
        ims.append(im_p)

    blank = numpy.ones(shape=(im.height, im.width, 3), dtype=numpy.uint8)*255
    im = PIL.Image.fromarray(blank, mode='RGB')
    im_p = im.quantize(palette=im0);
    ims.append(im_p)
            
    f = io.BytesIO()
    im0.save(f, 'gif', save_all=True, append_images=ims, duration=1000, loop=0)

    size = len(f.getbuffer())/1024;
    if (size > 3000):
        warnings.warn(f"Large GIF: {size} KiB")
    return IPython.display.display(IPython.display.Image(data=f.getvalue()))

## Equilibration challenges

In the previous section, you ran the hard sphere system for many time steps to **equilibrate** it and saved the trajectory in `trajectory.gsd`.
Is the final **state** you obtained actually an **equilibrium state**?
Statistical mechanics tells us that as long as our system is *ergodic* it will *eventually* achieve equilibrium.
You need to analyze this trajectory to determine if you have achieved the ordered **equilibrium** structure.

## Tools

There are many tools that can read **GSD** files and analyze or visualize the simulations, and many more Python packages that can work with the numerical data.
This tutorial will show you how to use **freud** to determine which particles are in a solid-like environment and **fresnel** to render system configurations.
The [freud](https://freud.readthedocs.io/) Python package provides a simple, flexible, powerful set of tools for analyzing trajectories obtained from molecular dynamics or Monte Carlo simulations.
The [fresnel](https://fresnel.readthedocs.io/) Python package produces publication quality renders with soft lighting, depth of field and other effects.

While outside the scope of this tutorial, you might want to use tools such as [OVITO](https://www.ovito.org/) or [VMD](https://www.ks.uiuc.edu/Research/vmd/) to visualize the trajectory interactively.
OVITO has built-in support for **GSD** files.
The [gsd-vmd](https://github.com/mphowardlab/gsd-vmd) plugin adds support to VMD.

## Read the trajectory

Use **GSD** to open the **trajectory** generated by the previous section of this tutorial.

In [ ]:
traj = gsd.hoomd.open('trajectory.gsd')

You can index into the frames of the trajectory like a list.
See how many frames exist in the trajectory:

In [ ]:
len(traj)

## Ergodicity

A system is *ergodic* when it can explore all possible states by making small moves from one to another.
In HPMC simulations, low volume fraction simulations are ergodic while very high volume fraction ones are not.
At high volume fractions, there isn't enough free space for the particles to rearrange so they get stuck in local configurations.

Visualize the motion of just a few particles to see if they appear stuck or if they are freely moving about the box:

In [ ]:
render_movie(traj[::20], particles=[1, 60])

Over the course of the short simulation, individual particles are able to move a few diameters.
This indicates that our system is able to explore phase space and is likely *ergodic*.

## Simulation length

How can you tell if you have run long enough to **equilibrate** the system?
The hard sphere system forms the fcc structure by nucleation and growth.
Nucleation is a rare event, so you need to keep running the simulation until it occurs.
If you ran this simulation many times with different random seeds, each would take a different number of steps to nucleate.
You need to examine the simulation trajectory in detail to determine if you have run it long enough.

In [ ]:
render_movie(traj[0:60:12])

Can you see the system ordering?
The particle positions rearrange to line up on evenly spaced planes.

You can use **freud's SolidLiquid** analysis method to quantitatively identify which particles are in the solid structure. Loop over all of the frames in the file and create a boolean array that indicates which particles are in a solid environment:

In [ ]:
solid = freud.order.SolidLiquid(l=6, q_threshold=0.7, solid_threshold=6)
is_solid = []
for frame in traj:
    solid.compute(system=(frame.configuration.box, frame.particles.position),
                  neighbors=dict(mode='nearest', num_neighbors=8))
    is_solid.append(solid.num_connections > solid.solid_threshold)

Plot the total number of particles in a solid environment over time:

In [ ]:
fig = matplotlib.figure.Figure(figsize=(10, 6.18))
ax = fig.add_subplot()
num_solid = [numpy.sum(a) for a in is_solid]
ax.plot(num_solid)
ax.set_xlabel('frame')
ax.set_ylabel('number of particles in a solid environment')
fig

This plot confirms what you saw visually and what you should expect in a system that nucleates and grows a crystal.
There is no solid at the beginning of the simulation.
Then a solid cluster forms and grows quickly to fill the box.

The next section puts the entire hard sphere simulation and analysis workflow together in one notebook so you can run it with different parameters and see the outcome.

[Previous section](06-Equilibrating-the-System.ipynb). [Next section](../01-Nematic-Transition-with-Spherocylinders/00-index.ipynb).